In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import matplotlib.animation as animation
import glob, re
import scienceplots

import scipy.sparse as sp
import scipy.sparse.linalg as spla


In [ ]:
# Show grid lines and ticks

Lx = 2.0
Ly = 2.0
Nx = 5
Ny = 5

hx = Lx/Nx
print(f'hx = {hx}')


fig, ax = plt.subplots(figsize=(8, 8))
for i in range(Nx+1):
    ax.axvline((1.0*i)/Nx*Lx, color='k', lw=0.5)
    print(f'({i/Nx*Lx}, 0) -> ({i/Nx*Lx}, {Ly})')
for j in range(Ny+1):
    ax.axhline(j/Ny*Ly, color='k', lw=0.5)
# ax.set_xticks(np.linspace(0, Lx, Nx+1))
# ax.set_yticks(np.linspace(0, Ly, Ny+1))
# ax.set_xlim(0, Lx)
# ax.set_ylim(0, Ly)
# ax.set_xlabel(r'$x$')
# ax.set_ylabel(r'$y$')
# ax.set_aspect('equal')
ax.set_axis_off()
# Show node numbering
delta_x = 0.004
delta_y = 0.02
for i in range(Nx+1):
    for j in range(Ny+1):
        ax.text(i/Nx*Lx+delta_x, j/Ny*Ly+delta_y, f'({i}, {j})->{i+j*(Nx+1)}', ha='left', va='center', fontsize=8)

fig.tight_layout()
plt.show()

In [ ]:
Lx = 2.0
Ly = 1.0
Nx = Ny = 50

hx = Lx / (Nx-1)
hy = Ly / (Ny-1)

# Matrix assembly for 5-point stencil '(Nx+1)*(Ny+1)' grid points.
A = sp.diags([1, 1, -4, 1, 1], [-Nx-1, -1, 0, 1, Nx+1], shape=((Nx+1)*(Ny+1), (Nx+1)*(Ny+1)), format='csr') 

# A[0, Nx*Nx-1] = -40

plt.imshow(A.todense(), vmin=-1, vmax=1, cmap='jet')


In [ ]:
A = sp.diags([1, 1, -4, 1, 1], [-Nx-1, -1, 0, 1, Nx+1], shape=((Nx+1)*(Ny+1), (Nx+1)*(Ny+1)), format='csr') / (hx*hx)


print(f"Shape of A: {A.shape}")
def fxy(x, y):
    return -1.0

# RHS assembly with forcing function f(x, y).
b = np.zeros((Nx+1)*(Ny+1))
for i in range(Nx):
    for j in range(Ny):
        b[i*Nx+j] = fxy((i+0.5)*hx, (j+0.5)*hy) * hx*hx

print(f"Shape of b: {b.shape}")

Nb = 0
x = 0

Anode1 = 0.5, 1.0
Anode2 = 1.5, 1.0
Cathode1 = 1.0, 1.0

# Dirichlet boundary conditions = 0
for idx_x in range(Nx+1):
    for idx_y in range(Ny+1):
        if idx_x == 0 or idx_x == Nx or idx_y == 0 or idx_y == Ny:
            line_idx = idx_x + idx_y*(Nx+1)
            A[line_idx, :] = 0
            A[line_idx, line_idx] = 1
            b[line_idx] = 0 if idx_x == 0 or idx_x == Nx else 1





# Dirichlet boundary conditions on lef


plt.imshow(A.todense(), vmin=-1, vmax=1)

# Solve linear system
u = spla.spsolve(A, b).reshape((Nx+1, Ny+1))

# Plot solution
fig, ax = plt.subplots()
im = ax.imshow(u, origin='lower', extent=[0, Lx, 0, Ly], cmap='jet')
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$y$')
ax.set_aspect('equal')
fig.colorbar(im, ax=ax)
plt.show()
# Min max
print(f'U min: {u.min():.3f}')
print(f'U max: {u.max():.3f}')




In [ ]:
# Streamlines
x = np.linspace(0, Lx, Nx+1)
y = np.linspace(0, Ly, Ny+1)
X, Y = np.meshgrid(x, y)
U, V = np.gradient(u, hx, hy)
fig, ax = plt.subplots()
ax.streamplot(X, Y, U, V, color='k', linewidth=0.5, density=5)
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$y$')
ax.set_aspect('equal')

In [ ]:


# Animate particles
fig, ax = plt.subplots()

ax.streamplot(X, Y, U, V, color='k', linewidth=0.5, density=5)
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$y$')
ax.set_aspect('equal')

scat = ax.scatter(X_Part, Y_Part, s=10, c='r')

# Particle animation
def init():
    return ax,

X_Part = np.random.rand(100)
Y_Part = np.random.rand(100)
print(X_Part)

def animate(i):
    NewX_Part = X_Part + U[np.round(X_Part/hx).astype(int), np.round(Y_Part/hy).astype(int)]*hx
    NewY_Part = Y_Part + V[np.round(X_Part/hx).astype(int), np.round(Y_Part/hy).astype(int)]*hy
    scat.set_offsets([NewX_Part, NewY_Part])
    X_Part = np.copy(NewX_Part)
    Y_Part = np.copy(NewY_Part)



anim = animation.FuncAnimation(fig, animate, init_func=init, frames=100, interval=10, blit=True, repeat=False)
anim.save("DDPD.mp4")